In [4]:
#kmeans clustering
#20BCE1143
#Advait (CSE3024 Web Mining Lab 6)

Import all required packages and modules

In [5]:
import requests
import bs4
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix
import time
import random

Define the two categories of text we will be classifying

In [6]:
#two categories of Fiction Discussion and Gaming
#from site https://forums.sufficientvelocity.com/forums 
#<note to self> increase categories to check clustering

gaming = "https://forums.sufficientvelocity.com/forums/gaming.8/page-"
fiction = "https://forums.sufficientvelocity.com/forums/fiction-discussion.10/page-"
versus = "https://forums.sufficientvelocity.com/forums/versus-debates.7/page-"
warpeace = "https://forums.sufficientvelocity.com/forums/war-peace.6/page-"
quests = "https://forums.sufficientvelocity.com/forums/quests.29/page-"
userfiction = "https://forums.sufficientvelocity.com/forums/user-fiction.2/page-"

categories = [gaming, fiction, versus, warpeace, quests, userfiction]

#each page has a list of threads with extensive discussion on each thread.
#can take the discussion into account as well, but navigation is an issue
#if only thread title, then sample size can become an issue

Create a function to return a list with all the titles of a particular category
We also use the function to iterate over a sample of webpages

Another method of doing this is to archive text from the over 300+ pages between the two categories and select a subset, but rate limiting is an issue

In [7]:
def returnTitles(pageNos, givenUrl):
    titles = []
    for pageNum in pageNos:
        #access the url by appending the page number to the end of the string
        url = givenUrl + str(pageNum)
        #print(url)
        response = requests.get(url)
        content = response.content
        soup = bs4.BeautifulSoup(content, "html.parser")
        
        curPageTitles = soup.findAll("div", class_="structItem-title")
        for title in curPageTitles:
            titles.append(title.find("a").text)
        
        #wait for a sec to stop overload
        time.sleep(1)
    
    return titles


code to create a train test dataset of randomly selected webpages

In [12]:
def sample_sets(n, m, k):
    numbers = random.sample(range(1, n+1), m)
    return numbers[:k], numbers[k:]

total_numbers = 100
sample_size = int(((40/len(categories))-(40/len(categories))%1))
test_size = 1

train_sets = []
test_sets = []

for category in categories:
    train, test = sample_sets(total_numbers, sample_size, sample_size-test_size)
    train_sets.append(train)
    test_sets.append(test)
    
# gaming_train, gaming_test = sample_sets(total_numbers, sample_size, sample_size - test_size)
# fiction_train, fiction_test = sample_sets(total_numbers, sample_size, sample_size - test_size)

print("Pages chosen for training are: ")
print(train_sets)

print("Pages chosen for testing are:")
print(test_sets)


Pages chosen for training are: 
[[28, 92, 53, 42, 40], [77, 97, 60, 85, 17], [96, 31, 77, 62, 51], [54, 32, 79, 6, 68], [87, 92, 42, 62, 19], [47, 34, 94, 26, 3]]
Pages chosen for testing are:
[[34], [98], [32], [16], [14], [63]]


run the code to obtain the various pages

Going to take a long while because I added the 1 sec sleep per page to be polite


In [6]:
#structId-title for title
#nav next for next page
#can go through every other page for a good time-wise distribution

gaming_titles = returnTitles(gaming_train, gaming)
fiction_titles = returnTitles(fiction_train, fiction)


Do the same for the test ones

In [7]:
gaming_test = returnTitles(gaming_test, gaming)
fiction_test = returnTitles(fiction_test, fiction)

Label data and combine to form a testing and training group

Shuffle or no shuffle doesnt matter

In [8]:
# Combine the titles and their respective labels (0 for gaming, 1 for fiction) into two arrays
train_data = gaming_titles + fiction_titles
train_target = [0] * len(gaming_titles) + [1] * len(fiction_titles)

test_data = gaming_test + fiction_test
test_target = [0] * len(gaming_test) + [1] * len(fiction_test)

pieline and prediction combined

In [9]:
def create_model(train_data, train_target, test_data, test_target):
    model = make_pipeline(TfidfVectorizer(), MultinomialNB())
    model.fit(train_data, train_target)
    labels = model.predict(test_data)
    return labels

In [10]:
# Train and test the model
labels = create_model(train_data, train_target, test_data, test_target)

Check accuracy with a confusion matrix

In [11]:
# Create the confusion matrix
confusion_matrix = confusion_matrix(test_target, labels)

# Print the confusion matrix
print("Confusion Matrix:")
print(confusion_matrix)

Confusion Matrix:
[[18 12]
 [ 9 21]]


Not great accuracy, shows that gaming and fiction might be pretty closely related as a takeaway
